In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from src.dataloaders_and_sets.simple_dataset import SimpleDataset
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch.optim as optim
from torcheval.metrics.functional import r2_score
from torch.optim.lr_scheduler import ExponentialLR
import tensorboard
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from src.models.fc import VAE
from torch.nn.functional import mse_loss
from src.training import variational_train

# Loading Data and Preprocessing

In [2]:
data_with_targets = pd.read_csv('data/data.csv', index_col=0)

In [3]:
data = data_with_targets.fillna(0.0)

In [4]:
# remove the possible y labels:
y_labels = ['primary_disease', 'gender', 'age', 'dataset']
data_columns = [col for col in data.columns if col not in y_labels]
y = "gender"
# TODO: makes no sense when using a autoencoder...
X_train, X_test, y_train, y_test = train_test_split(data[data_columns], data[y_labels], train_size=0.8, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=42)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(7539, 17137) (7539, 4) (1885, 17137) (1885, 4) (2356, 17137) (2356, 4)


In [5]:
# transform dataset for the simple autoencoder

transform_fc_ae = {
    "z_score": "per_sample",
    "most_variant": 5000,
}

In [6]:
if transform_fc_ae.get("most_variant") is not None:
    input_dim = transform_fc_ae.get("most_variant")
else: 
    input_dim = X_train.shape()[1]


In [8]:
# config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("{} is used".format(device))

cpu is used


# Setup Tensorboard

In [9]:
writer = SummaryWriter("logs/autoencoder")


In [10]:
hidden_one_dim = 512
hidden_two_dim = 128
z_dim = 64

num_epochs = 50
batch_size = 512
learning_rate = 3e-3
beta = 0.1
print(input_dim)

5000


run `tensorboard --logdir=./`

# Training

## Simple autoencoder

In [9]:
from src.models.fc import AE

batch_size = 512
learning_rate = 6e-4
num_epochs = 200

hidden_one_dim = 512
hidden_two_dim = 128
z_dim = 64


ae_dataset = SimpleDataset(X_train, transform=transform_fc_ae)
ae_dataloader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
ae = AE(input_size=input_dim, hidden_one_size=hidden_one_dim, hidden_two_size=hidden_two_dim, z_size=z_dim).to(device)
optimizer = optim.Adam(ae.parameters(), lr=learning_rate)
scheduler = ExponentialLR(optimizer, gamma=0.992)
loss_mse = nn.MSELoss()

ae_val_dataset = SimpleDataset(X_val, transform=transform_fc_ae)
ae_val_dataloader = DataLoader(ae_val_dataset, batch_size=batch_size, shuffle=True)


NameError: name 'input_dim' is not defined

In [ ]:
ae.eval()
writer.add_graph(ae, ae_dataset[0].to(device))
writer.close()


In [ ]:
# tensorboard logging
log_dir = "" + datetime.now().strftime("%Y%m%d-%H%M%S")
additional_comment = "just for checking what happens if"
writer = SummaryWriter(f"logs/autoencoder/{log_dir}", comment=f"Architecture=({input_dim}-{hidden_one_dim}-{hidden_two_dim}-{z_dim}), learning_rate={learning_rate}, batch_size={batch_size}, number epochs={num_epochs}" + additional_comment)

# training loop
for epoch in range(num_epochs):
    
    ae.train()
        
    for train_iteration, batch in enumerate(tqdm(ae_dataloader, desc="Training epoch {}".format(epoch+1))):
        batch = batch.to(device)
        x_reconstructed = ae(batch)
        loss = loss_mse(batch, x_reconstructed)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        writer.add_scalar("ae train loss", loss.item(), epoch * len(ae_dataloader) + train_iteration)
        score = r2_score(x_reconstructed, batch).item()
        writer.add_scalar("ae R2 Score", score, epoch * len(ae_dataloader) + train_iteration)
    
    scheduler.step()

    ae.eval()
    with torch.no_grad():
        for val_iteration, batch in enumerate(tqdm(ae_val_dataloader, desc="Validation epoch {}".format(epoch+1))):
            batch = batch.to(device)

            x_reconstructed = ae(batch)

            score = r2_score(x_reconstructed, batch).item()

            writer.add_scalar("ae validation R2Score", score, epoch * len(ae_val_dataloader) + val_iteration)

    
        
        

## Variational Autoencoder

In [43]:
batch_size = 512
learning_rate = 6e-4
num_epochs = 25
beta = 0.2

hidden_one_dim = 512
hidden_two_dim = 128
z_dim = 64


vae_dataset = SimpleDataset(X_train, transform=transform_fc_ae)
vae_dataloader = DataLoader(vae_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
vae = VAE(input_size=input_dim, hidden_one_size=hidden_one_dim, hidden_two_size=hidden_two_dim, z_size=z_dim).to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = ExponentialLR(optimizer, gamma=0.95)

vae_val_dataset = SimpleDataset(X_val, transform=transform_fc_ae)
vae_val_dataloader = DataLoader(ae_val_dataset, batch_size=batch_size, shuffle=True)


In [44]:
# tensorboard logging
log_dir = "" + datetime.now().strftime("%y%m%d-%H%M%S")
additional_comment = ""
writer = SummaryWriter(f"logs/variational autoencoder/{log_dir}_train", comment=f"Architecture=({input_dim}-{hidden_one_dim}-{hidden_two_dim}-{z_dim}), learning_rate={learning_rate}, batch_size={batch_size}, number epochs={num_epochs}, beta={beta}" + additional_comment)

writer_val = SummaryWriter(f"logs/variational autoencoder/{log_dir}_validation", comment=f"Architecture=({input_dim}-{hidden_one_dim}-{hidden_two_dim}-{z_dim}), learning_rate={learning_rate}, batch_size={batch_size}, number epochs={num_epochs}" + additional_comment)
vae.eval()
writer.add_graph(vae, vae_dataset[0].to(device))
writer.close()


for epoch in range(num_epochs):
    
    variational_train(vae, vae_dataloader, vae_val_dataloader, optimizer, scheduler, writer, writer_val, epoch, device, beta=beta)

/home/fes/Nextcloud/Uni/B.Sc. Bioinfo/Bachelorarbeit/thesis/src/dataloaders_and_sets/simple_dataset.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index], dtype=torch.float32)
/home/fes/anaconda3/envs/Helmholtz/lib/python3.11/site-packages/torch/jit/_trace.py:1116: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%eps : Float(64, strides=[1], requires_grad=0, device=cpu) = aten::rand_like(%std, %66, %67, %68, %69, %70) # /home/fes/Nextcloud/Uni/B.Sc. Bioinfo/Bachelorarbeit/thesis/src/models/fc.py:32:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  _check_trace(
/home/fes/anaconda3/envs/Helmholtz/lib/python3.11/site-packages/torch/jit/_trace.py:11

In [45]:
print(f"Architecture=({input_dim}-{hidden_one_dim}-{hidden_two_dim}-{z_dim}), learning_rate={learning_rate}, batch_size={batch_size}, number epochs={num_epochs}" + additional_comment)

Architecture=(5000-512-128-64), learning_rate=0.0006, batch_size=512, number epochs=25just for checking what happens if
